In [1]:
# Imports
import pandas as pd
import numpy as np
import nltk
import string
import re
import pickle

In [22]:
url = "https://uom-twitter-sentiment-analysis.s3.us-east-2.amazonaws.com/Twitter_sentiment_clean.csv"
data = pd.read_csv(url)

In [3]:
data.head(50)

,target,text
0,0,@chrishasboobs AHHH I HOPE YOUR OK!!!
1,0,"@misstoriblack cool , i have no tweet apps fo..."
2,0,@TiannaChaos i know just family drama. its la...
3,0,School email won't open and I have geography ...
4,0,upper airways problem
5,0,Going to miss Pastor's sermon on Faith...
6,1,on lunch....dj should come eat with me
7,0,@piginthepoke oh why are you feeling like that?
8,0,gahh noo!peyton needs to live!this is horrible
9,1,@mrstessyman thank you glad you like it! There...


data.text.str.contains('@')
data['text'] = data['text'].str.replace(r'@\S+', '')

In [4]:
# Remove punctuation
def remove_punct(text):
    new_text = []
    for t in text:
        if t not in string.punctuation:
            new_text.append(t)
    return ''.join(new_text)

In [21]:
# Tokenizer
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [6]:
# Removing stop words
from nltk.corpus import stopwords

def remove_sw(text):
    new_text = []
    for t in text:
        if t not in stopwords.words('english'):
            new_text.append(t)
    return new_text

In [7]:
# Lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer

def word_lemmatizer(text):
    new_text = []
    for t in text:
        lem_text = lemmatizer.lemmatize(t)
        new_text.append(lem_text)
    return new_text

In [23]:
df_1 = data.iloc[:10000,:]
df_1.head(50)

,target,text
0,0,@chrishasboobs AHHH I HOPE YOUR OK!!!
1,0,"@misstoriblack cool , i have no tweet apps fo..."
2,0,@TiannaChaos i know just family drama. its la...
3,0,School email won't open and I have geography ...
4,0,upper airways problem
5,0,Going to miss Pastor's sermon on Faith...
6,1,on lunch....dj should come eat with me
7,0,@piginthepoke oh why are you feeling like that?
8,0,gahh noo!peyton needs to live!this is horrible
9,1,@mrstessyman thank you glad you like it! There...


In [24]:
df_1['text'] = df_1['text'].apply(lambda t: remove_punct(t))
df_1['text'] = df_1['text'].apply(lambda t: tokenizer.tokenize(t.lower()))
df_1['text'] = df_1['text'].apply(lambda t: remove_sw(t))

/var/folders/4k/q9k3c4z91gxdn6zsywrgn8fw0000gn/T/ipykernel_79182/375162682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['text'] = df_1['text'].apply(lambda t: remove_punct(t))
/var/folders/4k/q9k3c4z91gxdn6zsywrgn8fw0000gn/T/ipykernel_79182/375162682.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['text'] = df_1['text'].apply(lambda t: tokenizer.tokenize(t.lower()))
/var/folders/4k/q9k3c4z91gxdn6zsywrgn8fw0000gn/T/ipykernel_79182/375162682.py:3: SettingWithCopyWarning: 
A value is tryin

In [26]:
data['text'] = data['text'].apply(lambda t: remove_punct(t))
data['text'] = data['text'].apply(lambda t: tokenizer.tokenize(t.lower()))
data['text'] = data['text'].apply(lambda t: remove_sw(t))

KeyboardInterrupt: 

In [ ]:
data['text'] = data['text'].apply(lambda t: word_lemmatizer(t))

In [25]:
df_1['text']

0                         [chrishasboobs, ahhh, hope, ok]
1             [misstoriblack, cool, tweet, apps, razr, 2]
2       [tiannachaos, know, family, drama, lamehey, ne...
3       [school, email, wont, open, geography, stuff, ...
4                               [upper, airways, problem]
                              ...                        
9995    [sister, cheyennes, birthday, party, today, go...
9996                                 [addieking, welcome]
9997    [rebekahmaylene, seriously, gals, making, nerv...
9998    [sarahmascara, thanks, want, chicken, cheeseca...
9999    [kutefans, sadly, probably, true, im, trying, ...
Name: text, Length: 10000, dtype: object